# 合并数据集的三种方法
>merge：根据一个或多个键将不同DataFrame中的行连接起来

>join

>concat：沿着一条轴将多个对象堆叠到一起

In [2]:
import pandas as pd
import numpy as np

## merge
merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, 
    suffixes=('_x', '_y'), copy=True, indicator=False)

>how : {'left', 'right', 'outer', 'inner'}, default 'inner'
>>其中：outer：并集     inner：交集     left/right：类似于SQL的join

>on：用于连接的列名，默认为交集

>left_on/right_on：指定左右DataFrame中用作连接键的列

>left_index/right_index：布尔值

>suffixes：用于追加到重叠列名的末尾

>indicator=True：可以注明每行数据的来源

### 普通合并
取键的交集
>pd.merge(df1,df2,on="key") 

取键的并集
>pd.merge(df1,df2,on="key",how="outer")  

对象列名不同，进行指定
>pd.merge(df1,df2,left_on="lkey",right_on="rkey")

In [4]:
#多对多的合并
df1 = pd.DataFrame({"key":["b","b","a","c","a","b"],"data1":range(6)})
df2 = pd.DataFrame({"key":["a","b","a","b","d"],"data2":range(5)})
pd.merge(df1,df2,on="key",how="left")

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


### 索引上的合并
利用left_index=True或者right_index=True
>pd.merge(left1,right1,left_on="key",right_index=True)

同时合并双方的索引
>pd.merge(left2,right2,how="outer",left_index=True,right_index=True)

## join
join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
>默认索引合并，当左右列名有重复时，需用lsuffix和rsuffix指定

merge和join只可以进行横向合并

## concatenate

### numpy中的concatenate

In [37]:
arr = np.arange(12).reshape(4,3)
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11]])

### pandas中的concatenate
concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)

>axis：默认纵向合并，axis=1可以实现横向合并

>join : {'inner', 'outer'}, default 'outer'

>join_axes：指明使用的索引

>keys：列表，注明合并后数组来源

>names：用于创建分层级别的名称

>ignore_index=True：不保留连接轴上的索引，产生一组新索引

In [7]:
s1 = pd.Series([0,1],index=["a","b"])
s2 = pd.Series([2,3,4],index=["c","d","e"])
pd.concat([s1,s2])

a    0
b    1
c    2
d    3
e    4
dtype: int64

# 合并重叠数据
>基于np.where

>或者combine_first()：组合Series值，优先第一个Series值，非NA值优先

In [9]:
df1 = pd.DataFrame([[1,np.nan]])
df2 = pd.DataFrame([[3,4]])
df1.combine_first(df2)

,0,1
0,1,4.0


# 重塑和轴向旋转

## 重塑层次化索引
>stack：列转行

>unstack：行转列

In [10]:
data = pd.DataFrame(np.arange(6).reshape(2,3),
                    index=pd.Index(["Ohio","Colorado"],name="state"),
                    columns=pd.Index(["one","two","three"],name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [11]:
data.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [12]:
data.unstack()

number  state   
one     Ohio        0
        Colorado    3
two     Ohio        1
        Colorado    4
three   Ohio        2
        Colorado    5
dtype: int32

## pivot
pivot(index=None, columns=None, values=None)
>透视，列转行

>values为填充DataFrame数据列的列名

In [13]:
frame = pd.DataFrame({'foo': ['one','one','one','two','two','two'],
                      'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                      'baz': [1, 2, 3, 4, 5, 6]})
frame.pivot(index="foo",columns="bar",values="baz")
#pivot相当于：set_index创建层次化索引，再用unstack重塑

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


## pivot_table()
透视表：根据一个或多个键对数据进行聚合，并根据行和列上的分组键将数据分配到各个矩形区域

pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=Non)

In [15]:
tips = pd.read_csv("D:/python/利用python进行数据分析/源码/ch08/tips.csv")
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [16]:
 tips.pivot_table(["tip","size"],index=["sex","day"],columns="smoker")

size                 tip          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   2.500000  2.000000  3.125000  2.682857
       Sat   2.307692  2.200000  2.724615  2.868667
       Sun   3.071429  2.500000  3.329286  3.500000
       Thur  2.480000  2.428571  2.459600  2.990000
Male   Fri   2.000000  2.125000  2.500000  2.741250
       Sat   2.656250  2.629630  3.256563  2.879259
       Sun   2.883721  2.600000  3.115349  3.521333
       Thur  2.500000  2.300000  2.941500  3.058000

In [17]:
#传入margins=True添加行和列的小计和总计
tips.pivot_table(["tip","size"],index=["sex","day"],columns="smoker",margins=True)

size                           tip                    
smoker             No       Yes       All        No       Yes       All
sex    day                                                             
Female Fri   2.500000  2.000000  2.111111  3.125000  2.682857  2.781111
       Sat   2.307692  2.200000  2.250000  2.724615  2.868667  2.801786
       Sun   3.071429  2.500000  2.944444  3.329286  3.500000  3.367222
       Thur  2.480000  2.428571  2.468750  2.459600  2.990000  2.575625
Male   Fri   2.000000  2.125000  2.100000  2.500000  2.741250  2.693000
       Sat   2.656250  2.629630  2.644068  3.256563  2.879259  3.083898
       Sun   2.883721  2.600000  2.810345  3.115349  3.521333  3.220345
       Thur  2.500000  2.300000  2.433333  2.941500  3.058000  2.980333
All          2.668874  2.408602  2.569672  2.991854  3.008710  2.998279

In [18]:
#传入其它函数
tips.pivot_table("tip",index=["sex","day"],columns="smoker",aggfunc="count",margins=True)

smoker        No  Yes  All
sex    day                
Female Fri     2    7    9
       Sat    13   15   28
       Sun    14    4   18
       Thur   25    7   32
Male   Fri     2    8   10
       Sat    32   27   59
       Sun    43   15   58
       Thur   20   10   30
All          151   93  244

## melt
逆透视：与pivot_table相反

melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None)
>id_vars：不需要被转换的列名

>value_vars：需要转换的列名；None对应其它列都需要被转化

>var_name：转化列的列名

>value_name：转化列对应的值

最终生成的dataframe：行数 = len(df)*len(value_vars)

In [21]:
df = pd.DataFrame({"A":{0:"a",1:"b",2:"c"},
                   "B":{0:1,1:3,2:5},
                   "C":{0:2,1:4,2:6}})
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [22]:
pd.melt(df,id_vars=["A"],value_vars=["B","C"])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [23]:
pd.melt(df,id_vars=["A"],value_vars=["B","C"],var_name="var",value_name="value")

,A,var,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


## pivot_table和melt的实际应用

melt数据特征：index很长，columns比较短

pivot_table数据特征：偏向于总体统计，生成m特征 * n特征的表格

melt只是改变原始数据的呈现形式，pivot_table则不一定（index相同的数据会被合并）,因此melt数据可以转化为pivot_table数据

示例：Seurat处理scRNA

原始结果

gene      cell  count       

33665     1      5

33663     1      4

33662     1     10

33660     1     11

33659     1     13

33658     1     10

33656     1     14

33655     1     39

33654     1     26

33653     1      8

33636     1      1

使用pivot_table进行转换
>matrix_df = matrix_df.pivot_table(["count"],index=["gene"],columns="cell")["count"]

                                      count  
cell   1     2     3     4     5     6    7     8     9     10    ...   5011  5012  5013  5014  5015  5016  5017  5018  
gene                                                           
4      NaN   NaN   NaN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN

5      NaN   NaN   NaN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN 

9      NaN   NaN   NaN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  ...    NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN 

10     NaN   NaN   NaN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN 

13     NaN   NaN   NaN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN 

15     NaN   NaN   NaN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

19     NaN   NaN   NaN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

## crosstab
交叉表：计算分组频率的特殊透视表

crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins=False, 
       margins_name='All', dropna=True, normalize=False)

In [24]:
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True)
#也可以使用pivot_table，但是不如crosstab方便

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

# 数据转化

## 移除重复数据

利用duplicated()返回布尔型Series
>筛选出非重复行：df[df.duplicated(["first_name","last_name"])==False]

>keep='last'可以让系统从后向前开始筛查，索引小的重复行会返回 'True'

利用drop_duplicates()

In [25]:
data = pd.DataFrame({"k1":["one"]*3+["two"]*4,"k2":[1,1,2,3,3,4,4]})
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [27]:
#指定部分列进行判断，保留第一个出现的值组合
data.drop_duplicates(["k1"])

,k1,k2
0,one,1
3,two,3


## 检测和过滤异常值
利用布尔型DataFrame的all和any方法

In [29]:
data = pd.DataFrame(np.random.randn(1000,4))
data[(np.abs(data)>3).any(1)]

,0,1,2,3
10,1.267400,-0.327953,-3.797830,0.162516
80,-0.576239,3.577904,0.335204,-0.432701
522,-0.292477,-0.399603,0.338472,-3.474212
590,-1.195342,-1.023822,-0.387939,-4.017866
758,-3.286389,-0.420194,-0.672075,-2.198840


# 删除特定行/列

## 删除所有值相同的列
>df.ix[:,(df!=df.ix[0]).any()]     
>>但单独时：df!=df.ix[0]为逐行处理

## 删除所有值相同的行
>df.ix[df.apply(lambda x:x!=x[0],axis=1).any(axis=1)]

## 删除存在0的列
>df.ix[:,df.all()]

## 删除存在0的行
>df.ix[df.all(axis=1)

## 删除所有值均为0的行
>df.ix[df.any(axis=1)]

# 排列和随机采样

In [31]:
#利用np.random.permutation：随机重排列
sample = np.random.permutation(5)
sample

array([4, 1, 2, 3, 0])

In [33]:
frame = pd.DataFrame(np.arange(5*4).reshape(5,4))
frame.take(sample)

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
0,0,1,2,3
